# 병원 입원기간 예측목적 : 입원 기간 동안 이용가능한 데이터와 이후 몇가지 테스트를 수행한 데이터를 사용하여환자의 입원 기간을 예측하는 머신러닝 모델로 입원 기간에 가장 많은 영향을 주는 요인이 무엇인지 밝히고,데이터로 부터 병원이 헬스케어 구조와 수익을 증진시키는데 도움이 되는 유용한 인사이트와 정책을 제시함

아래의 컬럼은 병원에 입원한 환자의 기록만 포함함.  * patientid : 환자 ID  * Age : 환자의 연령 범위  * gender : 환자의 성별  * Type of Admission : 입원 유형, 외상(Trauma) 이거나 비상(Emergency) 또는 긴급함(Urgent)  * Severity of illness : 질병의 심각도. 심함(Extreme), 중간(moderate) 또는 사소함(minor)  * health_conditions : 환자가 겪었던 과거 건강 컨디션  * Visitors with Patient : 환자와 동반한 환자들의 수  * Insurance : 환자가 건강보험이 있는지 없는지  * Admission_Deposit : 환자가 입원 기간동안 지불한 빚  * Stay (in days) : 환자의 입원기간 **(타겟변수)**  * Available Extra Rooms in Hospital : 입원기간 동안 이용가능한 방의 수  * Department : 환자를 돌보는 부서  * Ward_Facility_Code : 환자가 입원할 병동 시설의 코드  * doctor_name : 환자를 돌보는 의사  * staff_available: 병동에 현재 자리가 없는 직원의 수

# 문제 해결의 접근 방법  1. 필요 라이브러리 호출  2. 데이터셋 로드 및 관찰  3. 탐색적 데이터 분석 a.단변량, b.이변량  4. 필요시 데이터 전처리  5. 성능지표 정의 및 ML모델 구축  6. 가정 검토  7. 모델 비교 및 최적 모델 채택  8. 결론 및 제언

# Importing Libraries

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsimport warningswarnings.filterwarnings("ignore")# 디스플레이 되는 컬럼 제한 해제pd.set_option("display.max_columns", None)# 디스플레이 되는 200 행 으로 제한pd.set_option("display.max_rows", 200)from sklearn.model_selection import train_test_split, cross_val_score, KFoldfrom sklearn.linear_model import LinearRegression, Ridge, Lassofrom sklearn.tree import DecisionTreeRegressorfrom sklearn.ensemble import RandomForestRegressor, BaggingRegressorfrom sklearn.preprocessing import LabelEncoderfrom sklearn.model_selection import GridSearchCVfrom sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error

In [ ]:
data = pd.read_csv('./data/healthcare_data.csv')

In [ ]:
# 원본 데이터와의 변화를 피하기 위해 복제same_data = data.copy()

# Data Overview

In [ ]:
data.head()

,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,doctor_name,staff_available,patientid,Age,gender,Type of Admission,Severity of Illness,health_conditions,Visitors with Patient,Insurance,Admission_Deposit,Stay (in days)
0,4,gynecology,D,Dr Sophia,0,33070,41-50,Female,Trauma,Extreme,Diabetes,4,Yes,2966.408696,8
1,4,gynecology,B,Dr Sophia,2,34808,31-40,Female,Trauma,Minor,Heart disease,2,No,3554.835677,9
2,2,gynecology,B,Dr Sophia,8,44577,21-30,Female,Trauma,Extreme,Diabetes,2,Yes,5624.733654,7
3,4,gynecology,D,Dr Olivia,7,3695,31-40,Female,Urgent,Moderate,NaN,4,No,4814.149231,8
4,2,anesthesia,E,Dr Mark,10,108956,71-80,Male,Trauma,Moderate,Diabetes,2,No,5169.269637,34


In [ ]:
data.shape

In [ ]:
data.info()

**Observation**  * Avaiable Extra Rooms in Hospital, staff_available, Visitors with Patient, Admission_Deposit, Stay (in days) 은 수치형 데이터 타입이고, 나머지는 객체형 데이터 타입  * health_conditions 컬럼을 제외한 모든 컬럼은 결측치가 없다.  * patient_id는 식별자이다.

In [ ]:
def split_column_by_type(dataframe: pd.DataFrame):    cate_list, num_list = [], []    cate_list = dataframe.select_dtypes(include = ["object", "category"]).columns.tolist()    num_list = dataframe.select_dtypes(include = ["int", "float"]).columns.tolist()    print("other type columns : ",dataframe.drop(columns = cate_list + num_list).columns.tolist())    print(f'categorical columns : {cate_list} \nnumerical columns : {num_list}')    return cate_list, num_listcategorical_column_list, numerical_column_list = split_column_by_type(data)

In [ ]:
# checking for duplicate values in the Datadata.duplicated().sum()

In [ ]:
#To view patientid and the number of times they have visited the hospitaldata['patientid'].value_counts()

**Observation**  * 병원에 입원한 같은 환자 데이터 수는 최대 21 최소 1이다.

In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Available Extra Rooms in Hospital,500000.0,3.638800,2.698124,0.000000,2.000000,3.000000,4.000000,24.00000
staff_available,500000.0,5.020470,3.158103,0.000000,2.000000,5.000000,8.000000,10.00000
patientid,500000.0,63150.519058,41689.479956,-3269.000000,25442.000000,57864.000000,103392.000000,134400.00000
Visitors with Patient,500000.0,3.549414,2.241054,0.000000,2.000000,3.000000,4.000000,32.00000
Admission_Deposit,500000.0,4722.315734,1047.324220,1654.005148,4071.714532,4627.003792,5091.612717,10104.72639
Stay (in days),500000.0,12.381062,7.913174,3.000000,8.000000,9.000000,11.000000,51.00000


**Obeservation**  * 평균적으로 **3개의** 방이 이용가능하다 (3.63). 병원이 꽉 차서 이용가능한 방이 없을 때도 있고, (0.00) 이용가능한 최대 방의 갯수는 24개이다.  * 평균적으로 **5개의** 스태프들이 시로운 환자들을 다룰 수 있으나, 0이될 때도 있다다. 병원에서 이용가능한 최대 스태프 명수는 10명이다.  * 평균적으로 **3명** 의 방문자들이 환자들과 동행한다. 몇몇 환자들은 혼자서 오기도 하고 (최소값이 0), 매우 드문 케이스로 32명의 방문자들이 같이 온경우도 있다. 방문자의 수와 환자의 심각도 사이에 상관관계가 있는지 확인해보는 것은 흥미로울 것 같다.  * 평균적으로 입원을 위한 빚의 범위는 대략 **4722 달러** 이다. 그리고 최소는 1654 달러가 매번 입원마다 지불되었다.  * 환자들의 방문일 수는 **최소 3일에서 최대 51일** 까지 큰 범위로 분포되어있다. 변수에 아웃라이어가 있어 보이며, 중앙값은 9일이다.

In [ ]:
for column in categorical_column_list:    print(data[column].value_counts(normalize = True))    print("-"*50)

**Observations**  * 대다수 (대략 ~82% (= 56%+26.3%) )의 환자들이 중간 (Moderate) 또는 사소한 (Minor) 질병으로 입원하였다. 심한 질병이 중간에서 사소한 질병보다 덜 빈번하게 발생한다는 점에 의하면 이해가 간다.  * gynecology(산부인과) 부서가 병원에서 가장 많은 수의 환자들을 받았다. (~ 68%) 반면에, surgery (수술) 부서는 매우 적다. (~1%)  * A병동과 C병동은 가장 적은 수의 환자를 수용합니다 (~ 12%). 이는 아마도 심한 질병을 가진 환자들과 수술이 필요한 환자들을 위한 병동일 지도 모릅니다. 만약 이 병동에서 입원한다면 더 오래 머무는지 확인해 보는 것은 흥미로울 것 같습니다.  * 대다수 환자들은 21-50 대 (~ 75%) 연령대 그룹이며, 여성 환자 (~ 75%) 들입니다. 환자들의 대부분이 산부인과 부서라는 점이 이를 증명합니다.  * 환자들의 대부분은 트라우마 케이스로 병원에 입원했습니다. (~ 62%)  * 고혈압과 당뇨병이 가장 흔한 건강 컨디션입니다.

# Exploratory Data Anlaysis (EDA)

## 단변량 분석 (Univariate Analysis)

In [ ]:
# function to plot a boxplot and a histogram along the same scale.def histogram_boxplot(data, feature, figsize = (12,7), kde = False, bins = None):    """    Boxplot and histogram combined    data: dataframe    feature: dataframe column    figsize: size of figure (default (12,7))    kde: whether to the show density curve (default False)    bins: number of bins for histogram (default None)    """    f, (ax_box, ax_hist) = plt.subplots(        nrows = 2, # Number of rows of the subplot grid = 2        sharex = True, # x-axis will be shared among all subplots        gridspec_kw = {"height_ratios": (0.25,0.75)},        figsize = figsize    ) # creating the 2 subplots    sns.boxplot(        data = data, x= feature, ax = ax_box, showmeans=True, color = "violet"    ) # boxplot will be created band a star will indicate the mean value of the column    sns.histplot(        data = data, x = feature, kde= kde, ax = ax_hist, bins = bins, palette = "winter"    ) if bins else sns.histplot(        data = data, x = feature, kde= kde, ax = ax_hist    ) # For histogram    ax_hist.axvline(        data[feature].mean(), color = "green", linestyle = "--"    ) # Add mean to the histogram    ax_hist.axvline(        data[feature].median(), color = "black", linestyle = "-"    ) # Add median to the histogram

### Length of stay

In [ ]:
histogram_boxplot(data, "Stay (in days)", kde = True, bins = 30)

**Observations:**  * 매우 적은 환자들이 10일 이상 병동에 머물고, 40일 이후로는 거의 머물지 않는다. 이 이유는 아마도 대다수 환자들이 중간 또는 사소한 질병으로 입원했기 때문일 것이다.  * 분포의 피크점은 대부분의 환자들이 8-9일 정도 머문다는 사실을 보여준다.

### Admission deposit

In [ ]:
histogram_boxplot(data, "Stay (in days)", kde = True, bins = 30)

**Observation**  * 입원 비용의 분포는 거의 정규분포에 가까우며, 양 끝에 아웃라이어가 있다. 몇몇 환자들이 매우 많은 비용을 지불했고, 몇몇 환자들이 매우 적은 비용을 지불했다.

In [ ]:
histogram_boxplot(data, "Stay (in days)", kde = True, bins = 30)

**Observation**  * 방문자들 수의 분포는 오른쪽으로 높게 치우친 분포를 보인다.  * 2 또는 4명이 가장 빈번한 환자에 대한 방문자의 수이다.

## 이변량 분석 (Bivariate Analysis)

In [ ]:
import seaborn as snsimport matplotlib.pyplot as pltplt.figure(figsize = (15,7))sns.heatmap(data[numerical_column_list].corr(), annot = True, vmin = -1, vmax = 1, fmt = ".2f", cmap = "Spectral")

**Observations:**  * 위 히트맵은 변수들간의 상관관계가 없음을 보여준다.  * 연속 변수들은 타겟변수들과 (Stay (in days)) 상관관계가 없고, 이는 범주형 변수가 예측에 중요할 것을 암시한다.

In [ ]:
# function to plot stack bar plotsdef stacked_barplot(data, predictor, target):    """    Print the category counts and plot a stacked bar chart    data: dataframe    predictor: independent variable    target: target variable    """    count = data[predictor].nunique()    sorter = data[target].value_counts().index[-1]    tab1 = pd.crosstab(data[predictor], data[target], margins = True).sort_values(        by = sorter, ascending = False    )    print(tab1)    print("-" * 120)    tab = pd.crosstab(data[predictor], data[target], normalize = "index").sort_values(        by = sorter, ascending = False    )    tab.plot(kind = "bar", stacked = True, figsize = (count +1, 5))    plt.legend(        loc = "lower left",        frameon = False,    )    plt.legend(        loc = "upper left",        bbox_to_anchor = (1,1)    )    plt.show()

다양한 병동에 대한 입원일 수 분포를 확인해봅시다.

In [ ]:
sns.barplot(y = "Ward_Facility_Code", x = 'Stay (in days)', data = data)plt.show()

**Observation**  * 이전에 우리가 세웠던 가설이 맞는것 같다. 병동 A와 C의 환자가 더 오랜 기간동안 머무른다.

In [ ]:
stacked_barplot(data, "Ward_Facility_Code", "Department")

**Observation**  * 병동 B,D,F 는 산부인과(gynecology) 환자들로만 채워져있다.  * 병동 A,C.E는 다른 질병들의 환자들로 채워져 있고, 수술 환자들은 A 병동에만 있다.

In [ ]:
stacked_barplot(data, "Ward_Facility_Code", "Department")

**Observation** :  * 병동 A가 심각 수준의 환자들이 가장 많고, 그에 따라 병원에서 더 오랜 기간 머무는 것 같다. 그에따라 더 많은 스태프들과 자원들이 다른 병동과 비교하여 필요할 것 같다.  * 병동 F는 사소한 질병수준의 환자들이 가장 많고, E는 중간 수준의 환자들이 가장 많다.

연령대도 입원 기간을 찾는데 중요한 요인일것 같다. 동일한 방식으로 확인해보자

In [ ]:
sns.barplot(y = "Ward_Facility_Code", x = 'Stay (in days)', data = data)plt.show()

**Observation**  * 1-10 과 50-100 연령대 구간의 환자들이 가장 높은 입원일 수를 가지는 것으로 확인되었다. 이는 21-50 연령대의 대부분 환자들이 산부인과 부서로 입원을 하고, 1-10 과 50-100 연령대의 환자들은 더 심각한 질병으로 입원하기 때문일지도 모른다.

# Data Preparation for Model Building

  * 모델을 만드는 작업 이전에, 범주형 변수를 인코딩 해야한다.  * 독립변수와 종속변수를 분리한다.  * 학습용 데이터와 테스트용 데이터를 분리하여, 학습 데이터로 트레이닝 된 모델을 평가한다.

In [ ]:
# Creating dummy variables for the categorical columns# drop_first = True 속성은 중복값을 피하기 위해 사용됨data = pd.get_dummies(    data,    columns = data.select_dtypes(include = ["object", "category"]).columns.tolist(),    drop_first = True,    dtype = int)

In [ ]:
data

,Available Extra Rooms in Hospital,staff_available,patientid,Visitors with Patient,Admission_Deposit,Stay (in days),Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,Ward_Facility_Code_B,Ward_Facility_Code_C,Ward_Facility_Code_D,Ward_Facility_Code_E,Ward_Facility_Code_F,doctor_name_Dr John,doctor_name_Dr Mark,doctor_name_Dr Nathan,doctor_name_Dr Olivia,doctor_name_Dr Sam,doctor_name_Dr Sarah,doctor_name_Dr Simon,doctor_name_Dr Sophia,Age_11-20,Age_21-30,Age_31-40,Age_41-50,Age_51-60,Age_61-70,Age_71-80,Age_81-90,Age_91-100,gender_Male,gender_Other,Type of Admission_Trauma,Type of Admission_Urgent,Severity of Illness_Minor,Severity of Illness_Moderate,health_conditions_Diabetes,health_conditions_Heart disease,health_conditions_High Blood Pressure,health_conditions_Other,Insurance_Yes
0,4,0,33070,4,2966.408696,8,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
1,4,2,34808,2,3554.835677,9,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0
2,2,8,44577,2,5624.733654,7,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
3,4,7,3695,4,4814.149231,8,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,2,10,108956,2,5169.269637,34,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,4,2,43001,3,4105.795901,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0
499996,13,8,85601,2,4631.550257,11,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
499997,2,3,22447,2,5456.930075,8,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
499998,2,1,29957,2,4694.127772,23,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [ ]:
# patientid는 식별자이기 때문에 drop하고, 분석에 포함하지 않는다.data = data.drop(columns = ["patientid"])

In [ ]:
# 독립변수와 종속변수를 분리x = data.drop('Stay (in days)', axis = 1)y = data['Stay (in days)']

In [ ]:
# 학습데이터와 테스트 데이터 분리x_train, x_test , y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle = True, random_state = 1)

In [ ]:
# train 및 test 데이터 shape 확인print("Shape of Training set :", x_train.shape)print("Shape of Test set :", x_test.shape)

# Modeling Building  * sklearn의 RMSE, MAE, R2 등 회귀 모델 평가를 위해 다양한 지표들을 사용할 것이다.  * MAPE 와 adjusted R2 를 구하기 위한 함수도 정의할 것이다.  * Mean absolute percentage error (MAPE) 는 예측 정확도를 퍼센테이지로 구한 지표이며, 각 예측 값 에서 실제값을 차감하고 실제값을 나눈 수치의 평균으로 계산된다. 극단값이 없다면 잘 작동하며, 실제 값이 0 이면 none 이다.

In [ ]:
# function to compute adjusted R-squareddef adj_r2_score(predictors, targets, predictions):    r2 = r2_score(targets, predictions)    n = predictors.shape[0]    k = predictors.shape[1]    return 1- ((1- r2) * (n-1) / (n-k-1))# function to compute MAPEdef mape_score(targets, predictions):    return np.mean(np.abs(targets - predictions)/ targets) * 100# function to compute different metrics to check performance of a regression modeldef model_performance_regression(model, predictors, target):    """    Function to compute different metrics to check regression model performance    model:regressor    predictors: independent variables    target: dependent variables    """    # predictiong using the independent variables    pred = model.predict(predictors)    r2 = r2_score(target, pred) # to compute R-squared    adjr2 = adj_r2_score(predictors, target, pred) # to compute adjusted R-squared    rmse = np.sqrt(mean_squared_error(target, pred)) # to compute RMSE    mae = mean_absolute_error(target, pred) # to compute MAE    mape = mape_score(target, pred) # to compute MAPE    # creating a dataframe of metrics    df_perf = pd.DataFrame(        {            'RMSE': rmse,            'MAE': mae,            'R-squard': r2,            'Adj. R-squared': adjr2,            'MAPE': mape,        },        index = [0],    )    return df_perf# RMSEdef rmse(predictions, targets):    return np.sqrt(((targets- predictions)**2).mean())# MAPEdef mape(predictions, targets):    return np.mean(np.abs((targets- predictions)) / targets) * 100# MAEdef mae(predictions, targets):    return np.mean(np.abs(targets- predictions))# Model Performance on test and train datadef model_pref(olsmodel, x_train, x_test, y_train, y_test):    # In sample prediction    y_pred_train = olsmodel.predict(x_train)    y_observed_train = y_train    # Prediction on test data    y_pred_test = olsmodel.predict(x_test)    y_observed_test = y_test    print(        pd.DataFrame(            {                "DATA": ["Train", "Test"],                "RMSE": [                    rmse(y_pred_train, y_observed_train),                    rmse(y_pred_test, y_observed_test),                ],                "MAE": [                    mae(y_pred_train, y_observed_train),                    mae(y_pred_test, y_observed_test),                ],                "MAPE": [                    mape(y_pred_train, y_observed_train),                    mape(y_pred_test, y_observed_test),                ],            }        )    )

In [ ]:
import statsmodels.api as sm# Statsmodel api 는 constant를 default로 추가해주지 않기 때문에, 정확성을 위해 추가해야함x_train1 = sm.add_constant(x_train)x_test1 = sm.add_constant(x_test)# create the modelolsmodel1 = sm.OLS(y_train, x_train1).fit()# get the model summaryolsmodel1.summary()

OLS Regression Results 
 
 Dep. Variable: Stay (in days) R-squared: 0.843 
 
 
 Model: OLS Adj. R-squared: 0.843 
 
 
 Method: Least Squares F-statistic: 5.796e+04 
 
 
 Date: Wed, 31 Jul 2024 Prob (F-statistic): 0.00 
 
 
 Time: 11:58:06 Log-Likelihood: -1.0246e+06 
 
 
 No. Observations: 400000 AIC: 2.049e+06 
 
 
 Df Residuals: 399962 BIC: 2.050e+06 
 
 
 Df Model: 37 
 
 
 Covariance Type: nonrobust 
 
 
 
 
 coef std err t P>|t| [0.025 0.975] 
 
 
 const 19.8975 0.054 365.723 0.000 19.791 20.004 
 
 
 Available Extra Rooms in Hospital 0.0786 0.002 42.310 0.000 0.075 0.082 
 
 
 staff_available -0.0009 0.002 -0.588 0.556 -0.004 0.002 
 
 
 Visitors with Patient 0.0002 0.002 0.096 0.923 -0.004 0.005 
 
 
 Admission_Deposit -3.839e-05 4.78e-06 -8.028 0.000 -4.78e-05 -2.9e-05 
 
 
 Department_anesthesia 6.0822 0.029 210.111 0.000 6.025 6.139 
 
 
 Department_gynecology 0.4649 0.019 24.427 0.000 0.428 0.502 
 
 
 Department_radiotherapy -4.6216 0.037 -126.134 0.000 -4.693 -4.550 
 
 
 Department_surgery 9.6876 0.044 218.616 0.000 9.601 9.774 
 
 
 Ward_Facility_Code_B 0.2576 0.011 23.619 0.000 0.236 0.279 
 
 
 Ward_Facility_Code_C 0.4349 0.035 12.604 0.000 0.367 0.502 
 
 
 Ward_Facility_Code_D 0.2039 0.010 19.458 0.000 0.183 0.224 
 
 
 Ward_Facility_Code_E 0.3084 0.021 14.399 0.000 0.266 0.350 
 
 
 Ward_Facility_Code_F 0.0034 0.011 0.307 0.759 -0.018 0.025 
 
 
 doctor_name_Dr John 7.2650 0.031 234.360 0.000 7.204 7.326 
 
 
 doctor_name_Dr Mark 1.1957 0.033 36.451 0.000 1.131 1.260 
 
 
 doctor_name_Dr Nathan -0.2119 0.015 -13.959 0.000 -0.242 -0.182 
 
 
 doctor_name_Dr Olivia -0.3586 0.015 -24.577 0.000 -0.387 -0.330 
 
 
 doctor_name_Dr Sam 1.2844 0.038 33.912 0.000 1.210 1.359 
 
 
 doctor_name_Dr Sarah 0.8186 0.016 50.613 0.000 0.787 0.850 
 
 
 doctor_name_Dr Simon 6.1611 0.069 89.090 0.000 6.026 6.297 
 
 
 doctor_name_Dr Sophia 0.2168 0.020 10.606 0.000 0.177 0.257 
 
 
 Age_11-20 -12.7378 0.065 -194.584 0.000 -12.866 -12.609 
 
 
 Age_21-30 -10.4830 0.062 -168.463 0.000 -10.605 -10.361 
 
 
 Age_31-40 -13.4355 0.062 -216.956 0.000 -13.557 -13.314 
 
 
 Age_41-50 -13.5113 0.062 -216.833 0.000 -13.633 -13.389 
 
 
 Age_51-60 -0.3665 0.065 -5.650 0.000 -0.494 -0.239 
 
 
 Age_61-70 -0.4217 0.064 -6.591 0.000 -0.547 -0.296 
 
 
 Age_71-80 -0.2193 0.066 -3.342 0.001 -0.348 -0.091 
 
 
 Age_81-90 -0.0174 0.072 -0.243 0.808 -0.158 0.123 
 
 
 Age_91-100 -0.2599 0.112 -2.322 0.020 -0.479 -0.041 
 
 
 gender_Male 6.0065 0.038 157.327 0.000 5.932 6.081 
 
 
 gender_Other -0.0257 0.030 -0.845 0.398 -0.085 0.034 
 
 
 Type of Admission_Trauma -0.0667 0.012 -5.626 0.000 -0.090 -0.043 
 
 
 Type of Admission_Urgent 0.1502 0.018 8.300 0.000 0.115 0.186 
 
 
 Severity of Illness_Minor 0.0432 0.016 2.659 0.008 0.011 0.075 
 
 
 Severity of Illness_Moderate 0.0987 0.016 6.253 0.000 0.068 0.130 
 
 
 health_conditions_Diabetes -0.3512 0.023 -15.038 0.000 -0.397 -0.305 
 
 
 health_conditions_Heart disease 0.5943 0.027 21.861 0.000 0.541 0.648 
 
 
 health_conditions_High Blood Pressure -0.0367 0.017 -2.164 0.030 -0.070 -0.003 
 
 
 health_conditions_Other 0.0065 0.014 0.460 0.646 -0.021 0.034 
 
 
 Insurance_Yes -0.0107 0.012 -0.880 0.379 -0.034 0.013 
 
 
 
 
 Omnibus: 131769.607 Durbin-Watson: 1.998 
 
 
 Prob(Omnibus): 0.000 Jarque-Bera (JB): 708956.819 
 
 
 Skew: 1.495 Prob(JB): 0.00 
 
 
 Kurtosis: 8.796 Cond. No. 1.03e+16 
 
 Notes: [1] Standard Errors assume that the covariance matrix of the errors is correctly specified. [2] The smallest eigenvalue is 8.91e-20. This might indicate that there are strong multicollinearity problems or that the design matrix is singular.

In [ ]:
lin_reg_test = model_performance_regression(olsmodel1, x_test1, y_test)lin_reg_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,3.144055,2.155765,0.843028,0.842962,19.676966


  * R-squared 값이 0.84 이다.  * 결과 변수를 예측하기에 모든 변수들이 통계적으로 유의미하진 않다. 통계적으로 유의미한 변수들은 어떤 것인지 확인해보기 위해 p-value 값을 확인해볼 필요가 있다.**회귀모델 결과 해석**  1. **Adjusted R-squared** : 모델 적합도를 설명함     * R-squared 값은 0부터 1까지 범위에 있고, 값이 더 높을 수록 더 잘 적합되었다는 것을 의미함     * Adj R-squared 값은 0.84 이다.  2. **coef** : 모든것이 동일하다는 가정 하에 한단위의 변수를 변경했을 때 Y 값의 변화를 의미함  3. **std err** : 계수의 정확성 수준을 의미함     * 값이 낮을 수록 계수가 더 정확하다는 것을 의미함  4. **P > |t|** : p-value     * Pr(>|t|) : 각 독립 변수들에 대하여 귀무가설과 대립 가설에 대한 확률     * H_0 : 귀무가설 - 독립 변수들이 유의미하지 않다.     * H_a : 대립가설 - 독립 변수들이 유의미하다.     * p-value가 0.05보다 적으면 통계적으로 유의미하다고 간주된다.  5. **Confidence Interval** : 95% 가능도 하의 계수가 바뀔 수 있는 범위를 표시함.  * R-sqaured값과 Adjusted R-squared 값이 대략 84% 이기 때문에, 입원일수에 대한 분산을 설명할 수 있는 수준이 대략 84%정도 된다는 것을 설명해주는 좋은 모델을 만들었다는 명백한 지표임.  * 회귀모델의 유의성을 검증하고, 무의미한 변수를 제거해봄

In [ ]:
model_pref(olsmodel1, x_train1, x_test1, y_train,y_test)

Train 데이터와 Test 데이터의 RMSE가 크게 다르지 않다. 우리의 모델이 학습 데이터에 오버피팅 되지 않았다는 것을 뜻한다.Mean Absolute error (MAE)는 현재 모델이 환자의 입원일수를 예측하는데 테스트 데이터에 대하여 평균 오차가 2.15 일 정도 날 수 있다는 것을 뜻한다.RMSE와 MAE의 단위는 같다 - 이 케이스에서는 (일) 단위로 같다. 하지만 RMSE는 MAE보다 크다. 이유는 RMSE는 아웃라이어에 더 페널티를 가하기 때문이다.테스트 데이터의 Mean Absolte Percentage Error는 약 19% 정도이다.

### 다중공선성 확인VIF를 사용해서 다중공선성이 있는지 확인해볼 것이다.VIF 점수가 5 초과하면, 제거하거나 처리가 되어서 모든 변수들은 VIF 점수가 5 미만을 가지게 될것이다.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factordef checking_vif(train):    vif = pd.DataFrame()    vif["features"] = train.columns    # calculating VIF for each feature    vif["VIF"] = [        variance_inflation_factor(train.values, i) for i in range(len(train.columns))    ]    return vif

In [ ]:
print(checking_vif(x_train1))

In [ ]:
# drop p-value > 0.05 featuresinsignificant_columns = olsmodel1.pvalues[(olsmodel1.pvalues > 0.05)].indexprint('ueless_columns: ', insignificant_columns)x_train2 = x_train1.drop(columns = insignificant_columns, axis = 1)x_test2 = x_test1.drop(columns = insignificant_columns, axis = 1)

In [ ]:
print(checking_vif(x_train1))

Age 11-40 의 VIF가 14~ 30 범위에서 5~ 11 범위로 줄어들었음을 확인할 수 있다.

In [ ]:
olsmodel2 = sm.OLS(y_train, x_train2).fit()olsmodel2.summary()

OLS Regression Results 
 
 Dep. Variable: Stay (in days) R-squared: 0.843 
 
 
 Model: OLS Adj. R-squared: 0.843 
 
 
 Method: Least Squares F-statistic: 6.918e+04 
 
 
 Date: Wed, 31 Jul 2024 Prob (F-statistic): 0.00 
 
 
 Time: 11:58:40 Log-Likelihood: -1.0246e+06 
 
 
 No. Observations: 400000 AIC: 2.049e+06 
 
 
 Df Residuals: 399968 BIC: 2.050e+06 
 
 
 Df Model: 31 
 
 
 Covariance Type: nonrobust 
 
 
 
 
 coef std err t P>|t| [0.025 0.975] 
 
 
 const 19.8763 0.038 523.273 0.000 19.802 19.951 
 
 
 Available Extra Rooms in Hospital 0.0786 0.002 42.394 0.000 0.075 0.082 
 
 
 Admission_Deposit -3.844e-05 4.77e-06 -8.053 0.000 -4.78e-05 -2.91e-05 
 
 
 Department_anesthesia 6.0820 0.029 210.268 0.000 6.025 6.139 
 
 
 Department_gynecology 0.4662 0.021 22.484 0.000 0.426 0.507 
 
 
 Department_radiotherapy -4.6222 0.037 -126.353 0.000 -4.694 -4.551 
 
 
 Department_surgery 9.6831 0.043 222.985 0.000 9.598 9.768 
 
 
 Ward_Facility_Code_B 0.2543 0.015 16.656 0.000 0.224 0.284 
 
 
 Ward_Facility_Code_C 0.4347 0.034 12.609 0.000 0.367 0.502 
 
 
 Ward_Facility_Code_D 0.2006 0.015 13.594 0.000 0.172 0.230 
 
 
 Ward_Facility_Code_E 0.3084 0.021 14.435 0.000 0.267 0.350 
 
 
 doctor_name_Dr John 7.2518 0.027 269.523 0.000 7.199 7.305 
 
 
 doctor_name_Dr Mark 1.1932 0.033 36.580 0.000 1.129 1.257 
 
 
 doctor_name_Dr Nathan -0.2109 0.015 -13.956 0.000 -0.240 -0.181 
 
 
 doctor_name_Dr Olivia -0.3577 0.015 -24.314 0.000 -0.387 -0.329 
 
 
 doctor_name_Dr Sam 1.2820 0.038 33.884 0.000 1.208 1.356 
 
 
 doctor_name_Dr Sarah 0.8179 0.016 50.884 0.000 0.786 0.849 
 
 
 doctor_name_Dr Simon 6.1509 0.069 89.713 0.000 6.016 6.285 
 
 
 doctor_name_Dr Sophia 0.2170 0.021 10.541 0.000 0.177 0.257 
 
 
 Age_11-20 -12.7256 0.042 -306.350 0.000 -12.807 -12.644 
 
 
 Age_21-30 -10.4708 0.036 -288.736 0.000 -10.542 -10.400 
 
 
 Age_31-40 -13.4232 0.036 -375.425 0.000 -13.493 -13.353 
 
 
 Age_41-50 -13.4991 0.036 -370.478 0.000 -13.571 -13.428 
 
 
 Age_51-60 -0.3542 0.041 -8.722 0.000 -0.434 -0.275 
 
 
 Age_61-70 -0.4095 0.039 -10.386 0.000 -0.487 -0.332 
 
 
 Age_71-80 -0.2071 0.042 -4.967 0.000 -0.289 -0.125 
 
 
 Age_91-100 -0.2478 0.100 -2.474 0.013 -0.444 -0.051 
 
 
 gender_Male 6.0074 0.036 165.106 0.000 5.936 6.079 
 
 
 Type of Admission_Trauma -0.0667 0.012 -5.623 0.000 -0.090 -0.043 
 
 
 Type of Admission_Urgent 0.1502 0.018 8.302 0.000 0.115 0.186 
 
 
 Severity of Illness_Minor 0.0435 0.016 2.677 0.007 0.012 0.075 
 
 
 Severity of Illness_Moderate 0.0991 0.016 6.283 0.000 0.068 0.130 
 
 
 health_conditions_Diabetes -0.3527 0.023 -15.296 0.000 -0.398 -0.308 
 
 
 health_conditions_Heart disease 0.5934 0.027 22.063 0.000 0.541 0.646 
 
 
 health_conditions_High Blood Pressure -0.0379 0.017 -2.250 0.024 -0.071 -0.005 
 
 
 
 
 Omnibus: 131773.426 Durbin-Watson: 1.998 
 
 
 Prob(Omnibus): 0.000 Jarque-Bera (JB): 708970.927 
 
 
 Skew: 1.495 Prob(JB): 0.00 
 
 
 Kurtosis: 8.796 Cond. No. 1.44e+16 
 
 Notes: [1] Standard Errors assume that the covariance matrix of the errors is correctly specified. [2] The smallest eigenvalue is 4.5e-20. This might indicate that there are strong multicollinearity problems or that the design matrix is singular.

In [ ]:
lin_reg_test = model_performance_regression(olsmodel1, x_test1, y_test)lin_reg_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,3.14407,2.155771,0.843027,0.842972,19.676973


In [ ]:
model_pref(olsmodel2, x_train2, x_test2, y_train,y_test)

**Observation**  * 학습데이터와 테스트데이터의 RMSE, MAE, MAPE 가 차이가 거이 없으므로, 오버피팅 되지 않고 잘 일반화가 되었음을 알 수 있다.

# Checking for assumption and rebuilding the model이번 단계에선 모델의 가정을 확인해보고, 가정에 문제가 있다면, 문제를 해결하고 모델을 재구축해보자.  1. 잔차평균은 0이어야 함.  2. 이분산성이 아니어야함 (등분산성)  3. 변수들의 선형성  4. 오차항의 정규성

## Mean of residuals should be 0 and normality of error terms

In [ ]:
residual = olsmodel2.resid # resid

In [ ]:
residual.mean()

잔차 평균이 0에 매우 근접하다. 그에 따라, 가정이 만족된다.

## Test for Normality**정규성검증이란?**  * 오차항/잔차가 정규분포되어야 한다.  * 오차항이 정규분포가 아니라면, 신뢰구간이 너무 넓거나 좁게된다. 한번 신뢰구간이 불안정하게 되면, 최소제곱합 근거로 계수를 추정하기 어려워진다.**비정규성이 암시하는 것은?**  * 비정상적인 몇몇 데이터 포인트가 있다는것을 암시하고, 더 좋은 모델을 위해 면밀히 연구될 필요가 있다는 것을 의미한다.**정규성을 어떻게 확인할것인가?**  * QQ plot을 통해 확인할 수 있다. 정규분포를 따르는 잔차는 일직선의 그림을 그릴 것이고, 정규분포를 따르지 않다면 일직선 그림이 그려지지 않을 것이다.  * 정규성을 검증하는 다른 방법을 통해 할 수 있다. : Shapiro-Wilk test**잔차가 비정상적이라는 것은 무엇인가?**  * log, exponential, arcsinh 등 데이터에 따라 변형을 가할 수 있다.

In [ ]:
sns.histplot(residual, kde= True)

잔차가 정규분포에 가까운것 같다. 정규성 가정이 만족된다.

## Linearity of Variables예측변수가 반드시 종속변수와 선형 관계를 가진다는 것을 의미한다.가정을 검증하기 위해, 잔차와 피팅된 값들간의 그림을 그려보고, 잔차들이 강한 패턴을 가지지 않는지 확인할 필요가 있다. x 축에 무작위로 균등하게 흩어져있어야 한다.

In [ ]:
fitted = olsmodel2.fittedvaluessns.residplot(x = fitted, y = residual, color = "lightblue")plt.xlabel("Fitted Values")plt.ylabel("Residual")plt.title("Residual PLOT")plt.show()

**Observation**  * 잔차와 적합된 값 사이에 패턴이 보이지 않는것을 확인할 수 있다. 즉 선형성 가정이 만족된다.

## No Heteroscedasticity### Test for Homoscedasticity  * **Homoscedasticity** (등분산성) - 잔차의 분산이 회귀선에 따라 체계적으로 분포되어 있다면, 데이터는 등분산성을 가진다고 한다.  * **Heteroscedasticity** (이분산성) - 잔차의 분산이 회귀선에 따라 동등하지 않다면, 데이터는 이분산성을 가진다고 한다. 이경우에 잔차는 화살표 모양이나 체계적이지 않은 모양을 가질 수 있다.등분산성을 검증하기 위해 Goldfeld-Quandt test를 사용할 것이다.귀무가설 : 잔차가 등분산성이다.대립가설 : 잔차가 이분산성이다.alpha = 0.05

In [ ]:
import statsmodels.stats.api as smfrom statsmodels.compat import lzipname = ["F statistic", "p-value"]test = sm.het_goldfeldquandt(residual, x_train2)lzip(name, test)

p-value가 0.05보다 크므로, 귀무가설을 기각하지 못하고 잔차가 등분산성을 가진다고 볼 수 있다.

In [ ]:
coef = olsmodel2.paramscoef

In [ ]:
Equation = "Stay (in days)="print(Equation, end = '\t')for i in range(len(coef)):    print('(', round(coef[i],2), ') *', coef.index[i], '+', end = ' ')

In [ ]:
coef.sort_values(ascending = False)

**Observation**  * 수술, Dr John, Dr simon, anesthesia(마취), 남자 일수록 병원에 오래 머물고  * 연령대가 21-50, 방사선치료일수록 병원에 짧게 머뭄

In [ ]:
orig_data = pd.read_csv('./data/healthcare_data.csv')

In [ ]:
orig_data.loc[(orig_data['Department'] == 'surgery'),'doctor_name'].value_counts()

# Q  * surgery 담당 의사는 Simon 과 Isaac 인데, John은 어떤 이유로 입원일수가 길도록 양의 계수가 나왔을까?

In [ ]:
sns.histplot(residual, kde= True)

In [ ]:
orig_data.loc[(orig_data['Department'] == 'surgery'),'doctor_name'].value_counts()

In [ ]:
orig_data.loc[(orig_data['Department'] == 'surgery'),'doctor_name'].value_counts()

In [ ]:
orig_data.loc[(orig_data['Department'] == 'surgery'),'doctor_name'].value_counts()

In [ ]:
sns.barplot(y = "Ward_Facility_Code", x = 'Stay (in days)', data = data)plt.show()

In [ ]:
sns.boxplot(y = "doctor_name", x = "Stay (in days)", data = orig_data)plt.show()

John의 계수가 왜 높게 나왔을까?

In [ ]:
stacked_barplot(orig_data, "doctor_name", "Type of Admission")

In [ ]:
stacked_barplot(orig_data, "doctor_name", "Type of Admission")

In [ ]:
sns.histplot(residual, kde= True)

In [ ]:
stacked_barplot(orig_data, "doctor_name", "Type of Admission")

In [ ]:
stacked_barplot(orig_data, "doctor_name", "Type of Admission")

In [ ]:
for doctor_name in orig_data['doctor_name'].unique():    sns.boxplot(data = orig_data[(orig_data['doctor_name'] == doctor_name)], x = 'Stay (in days)', hue = 'Department' )    plt.title(doctor_name)    plt.show()

In [ ]:
sns.boxplot(y = "doctor_name", x = "Stay (in days)", data = orig_data)plt.show()

# ASurgery 다음으로 anesthesia (마취), TB & Chest disease , radiotherapy 순으로 입원기간이 긴데, Dr John이 수술 외에 3가지 department를 모두 다하고 있어서 Dr John이 입원일수를 높게하는 변수로 도출된 것으로 확인

## cross validation 기법으로 다른 평가 지표로 평가해보고 모델 성능을 향상시키기

In [ ]:
from sklearn.model_selection import cross_val_scorelinearregression = LinearRegression()cv_Score11 = cross_val_score(linearregression, x_train2, y_train, cv = 10) # cv=10 :은 10개 덩어리로 데이터가 구분되었다는 것을 의미함cv_Score12 = cross_val_score(linearregression, x_train2, y_train, cv = 10                             , scoring = 'neg_mean_squared_error')print("RSquared: %0.3f (+/- %0.3f)" % (cv_Score11.mean(), cv_Score11.std() * 2))print("Mean Squared Error: %0.3f (+/- %0.3f)" % ( -1*cv_Score12.mean(), cv_Score12.std() * 2))

**Observation**  * 교차검증을 통한 R-squared는 0.843 이고, 학습 데이터의 R-squared도 0.843 이다.  * 교차검증을 통한 MSE는 9.831 이고, 학습데이터의 MSE도 9.83 이다.교차 검증을 통해, R-squared 값을 올리고, MSE를 줄이는 피쳐 엔지니어링이나 새로운 피쳐로 모델 구축 과정을 재반복하고 싶을지도 모른다.

## Ridge Regression

In [ ]:
rdg = Ridge()rdg.fit(x_train, y_train)model_pref(rdg, x_train, x_test, y_train, y_test)

In [ ]:
ridge_regression_perf_test = model_performance_regression(rdg, x_test, y_test)ridge_regression_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,3.144057,2.155826,0.843028,0.842963,19.677968


In [ ]:
lasso = Lasso()lasso.fit(x_train, y_train)model_pref(lasso, x_train, x_test, y_train, y_test)

In [ ]:
lasso_regression_perf_test = model_performance_regression(lasso, x_test, y_test)lasso_regression_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,6.064339,3.873332,0.416006,0.415766,34.652716


Ridge regression은 Linear Regression 과 비교햇을 때 비슷한 결과를 보인다.선형 모델을 사용하여 관계를 완전히 모델링할 수 없다는 느낌이 있기 때문에 모델의 성능을 향상시킬 수 있는 범위는 분명히 있는것 같다.이제 디시젼트리 Regressor 나 랜덤포레스트 Regressor 같은 비-선형 회귀모델을 만들어 보고 성능을 확인해보자(여기서 비선형 모델 학습에는 p-value가 0.05 이하인 변수들을 cutting 하지 않고 원 변수를 그대로 사용해본다. 이유는 선형모델에서는 다중공선성 문제를 해결하기 위해 p-value가 0.05보다 큰 변수들을 제거하였지만, 비선형모델에서는 선형모델의 가정을 만족시키는지 확인할 필요가 없기 때문이다. )

## Decision Tree Regressor

In [ ]:
# Decision Tree Regressordt_regressor = DecisionTreeRegressor(random_state = 1)# Fitting the modeldt_regressor.fit(x_train, y_train)# Model Performance on test data i.e predictiondt_regressor_perf_test = model_performance_regression(dt_regressor, x_test, y_test)dt_regressor_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,1.821321,1.13127,0.947324,0.947302,9.353216


## Tuning the Decision Tree Regressor

In [ ]:
# Choose the type of regressordtree_tuned = DecisionTreeRegressor(random_state = 1)# Grid of parameters to choose fromparameters = {    'max_depth': np.arange(2,8),    'criterion' : ['squared_error', 'friedman_mse'],    'min_samples_leaf' : [1, 3, 5, 7],    'max_leaf_nodes' : [2, 5, 7] + [None]}scorer = make_scorer(r2_score)# Run the grid searchgrid_obj = GridSearchCV(dtree_tuned, parameters, scoring = scorer, cv = 5, verbose = 1)grid_obj.fit(x_train, y_train)# Set the dtree_tuned_regressor to the best combination of parametersdtree_tuned_regressor = grid_obj.best_estimator_dtree_tuned_regressor.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=7, random_state=1) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. DecisionTreeRegressor DecisionTreeRegressor(max_depth=7, random_state=1)

In [ ]:
#Get the score of tuned decision tree regressordtree_tuned_regressor_perf_test = model_performance_regression(dtree_tuned_regressor, x_test, y_test)dtree_tuned_regressor_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,1.741743,1.161072,0.951826,0.951807,9.64668


튜닝된 디시젼 트리 모델의 피쳐 중요도를 확인해보자

In [ ]:
features = list(x.columns)importances = dtree_tuned_regressor.feature_importances_indices = np.argsort(importances)plt.figure(figsize = (10,10))plt.title("Feature Importance")plt.barh(range(len(indices)), importances[indices], color = 'violet', align = 'center')plt.yticks(range(len(indices)), [features[i] for i in indices])plt.xlabel('Relative Importances')plt.show()

**Observation**  * Department_gynecology, Age_41_50, Age_31_40 이 가장 중요한 피쳐들이다. 그 다음은 Department_anesthesia, Department_surgery 이다.  * 나머지 변수들은 병원 입원 기간을 결정하는데 영향을 주지 않았다.

## Bagging Regressor

In [ ]:
# Bagging Regressor# - 기존 학습 데이터(Original Data)로부터 랜덤하게 '복원추출'하여# - 동일한 사이즈의 데이터셋을 여러개 만들어 앙상블을 구성하는 여러 모델을 학습시키는 방법bagging_estimator = BaggingRegressor(random_state = 1)bagging_estimator.fit(x_train, y_train)bagging_estimator_perf_test = model_performance_regression(bagging_estimator, x_test, y_test)bagging_estimator_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,1.364505,0.902326,0.970434,0.970422,7.627444


### **Tuned Bagging Regressor**

In [ ]:
# Choose the type of regressor.bagging_tuned = BaggingRegressor(random_state=1)# Grid of parameters to choose fromparameters = {"n_estimators": [10,15,20],              "max_samples" : [0.8,1],              "max_features" : [0.8,1]             }# Type of scoring used to compare parameter combinationsscorer = make_scorer(r2_score)# Run the grid searchgrid_obj = GridSearchCV(bagging_tuned , parameters, scoring=scorer,cv=5)grid_obj = grid_obj.fit(x_train,y_train)# Set the bagging_tuned_regressor to the best combination of parametersbagging_tuned_regressor = grid_obj.best_estimator_bagging_tuned_regressor.fit(x_train, y_train)

BaggingRegressor(max_features=0.8, max_samples=0.8, n_estimators=15,
 random_state=1) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. BaggingRegressor BaggingRegressor(max_features=0.8, max_samples=0.8, n_estimators=15,
 random_state=1)

In [ ]:
#Get the score of tuned Bagging tree regressorbagging_tuned_regressor_perf_test = model_performance_regression(bagging_tuned_regressor, x_test, y_test)bagging_tuned_regressor_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,1.537709,1.033984,0.962452,0.962436,8.965645


## Random Forest Regressor(bagging vs random forest) <https://stats.stackexchange.com/questions/264129/what-is-the-difference-between-bagging-and-random-forest-if-only-one-explanatory>

In [ ]:
# Random Forest Regressor# - 의사결정나무를 배깅 방식으로 만든 알고리즘으로, 부트스트랩을 통해 다양한 서브 데이터셋을 생성하고 (의사결정 나무의 피쳐가 달라짐)# - 여러 개의 의사결정나무가 각각의 데이터셋을 학습하고 결과를 취합 함으로써 단일 의사결정나무가 가질 수 있는 과적합 문제를 해결할 수 있는 알고리즘regressor = RandomForestRegressor(random_state = 1)regressor.fit(x_train, y_train)regressor_perf_test = model_performance_regression(regressor, x_test, y_test)regressor_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,1.302336,0.863677,0.973067,0.973056,7.306138


## Tuned Random Forest Regressor**NOTE:** : 데이터 관측치 수가 크기 때문에, GridSearch 하는데 시스템 환경에 따라 1-2 시간정도 소요될 수 있다.

In [ ]:
rf_tuned = RandomForestRegressor(random_state = 1)# Grid of parameters to choose fromparameters = {    'n_estimators': [110, 120],    'max_depth': [5, 7],    'max_features': [0.8, 1],}# Type of scoring used to compare parameter combinationsscorer = make_scorer(r2_score)# Rund the grid searchgrid_obj = GridSearchCV(rf_tuned, parameters, scoring = scorer, cv = 5)grid_obj.fit(x_train, y_train)# Set the rf_tuned_regressor to the best combination of parametersrf_tuned_regressor = grid_obj.best_estimator_rf_tuned_regressor.fit(x_train, y_train)

RandomForestRegressor(max_depth=7, max_features=0.8, n_estimators=120,
 random_state=1) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. RandomForestRegressor RandomForestRegressor(max_depth=7, max_features=0.8, n_estimators=120,
 random_state=1)

In [ ]:
# Model Performance on test data i.e predictionrf_tuned_regressor_perf_test = model_performance_regression(rf_tuned_regressor, x_test, y_test)rf_tuned_regressor_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,1.723197,1.157816,0.952847,0.952827,9.66538


**Observations:**  * 튜닝된 모델의 성능은 디폴트 파라미터로 만들어진 모델과 비교했을 때 실제로 감소한 것이로 보인다.  * 계산상의 한계로 인해 극소수의 하이퍼파라미터와 소수의 값을 시도했기 때문일 것이다. 우리는 모델을 더 조정함으로써 모델의 성능을 향상시키려고 확실히 시도할 수 있다.

**feature importance 시각화**

In [ ]:
# Plotting the feature importanceimportances = rf_tuned_regressor.feature_importances_indices = np.argsort(importances)plt.figure(figsize = (10,10))plt.title('Feature Importances')plt.barh(range(len(indices)), importances[indices], color = 'violet', align = 'center')plt.yticks(range(len(indices)), [features[i] for i in indices])plt.xlabel('Relative Importances')plt.show()

**Observation**  * 3개의 중요 변수들은 트리 모델과 동일하다. gender, doctors 과 같은 모델에 추가된 다른 변수들도 있다.

In [ ]:
models_test_comp_df = pd.concat(    [        lin_reg_test.T,        ridge_regression_perf_test.T,        dt_regressor_perf_test.T,        regressor_perf_test.T,        bagging_estimator_perf_test.T,        dtree_tuned_regressor_perf_test.T,        bagging_tuned_regressor_perf_test.T,        rf_tuned_regressor_perf_test.T    ],    axis = 1)models_test_comp_df.columns = [    "Linear Regression",    "Ridge Regression",    "Decision tree regressor",    "Random Forest regressor",    "Bagging regressor",    "Tuned Decision Tree regressor",    "Tuned Bagging Tree regressor",    "Tuned Random Forest Regressor"]print("Test performance comparision:")models_test_comp_df

,Linear Regression,Ridge Regression,Decision tree regressor,Random Forest regressor,Bagging regressor,Tuned Decision Tree regressor,Tuned Bagging Tree regressor,Tuned Random Forest Regressor
RMSE,3.144070,3.144057,1.821321,1.302336,1.364505,1.741743,1.537709,1.723197
MAE,2.155771,2.155826,1.131270,0.863677,0.902326,1.161072,1.033984,1.157816
R-squard,0.843027,0.843028,0.947324,0.973067,0.970434,0.951826,0.962452,0.952847
Adj. R-squared,0.842972,0.842963,0.947302,0.973056,0.970422,0.951807,0.962436,0.952827
MAPE,19.676973,19.677968,9.353216,7.306138,7.627444,9.646680,8.965645,9.665380


**Observation**  * 모든 모델들이 RMSE와 R-squared 관점에서 linear 와 ridge 회귀와 비교했을 때 좋은 성능을 보인다.  * 배깅과 랜덤포레스트 모델은 단일 의사결정 모델보다 더 좋은 성능을 보인다.  * 디폴트 파라미터를 사용한 랜덤포레스트 모델이 모든 학습된 모델들 사이에 가장 좋은 성능을 보인다.

## Fitting the chosen final model  * 마지막 모델 구축 - 랜덤포레스트 Regressor  * 우리는 디폴트 파라미터를 가진 랜덤포레스트 Regressor 를 마지막 최종 모델로 고려할 것이다.

In [ ]:
final_model = RandomForestRegressor(n_estimators = 100, random_state = 1)final_model.fit(x_train, y_train)

RandomForestRegressor(random_state=1) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. RandomForestRegressor RandomForestRegressor(random_state=1)

In [ ]:
final_model_perf_train = model_performance_regression(final_model, x_train, y_train)

In [ ]:
final_model_perf_train

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,0.48724,0.320675,0.996203,0.996203,2.716192


  * **R-squared 와 Adjust R-sqaured 둘다 학습데이터에 대해 대략 99.6% 정도 된다.** 이것은 독립 변수를 사용하여 타겟 변수에 대한 거의 대부분의 분산을 설명한다고 암시할 수 있다.  * 테스트 데이터에 대해 빠르게 성능을 확인해보자

In [ ]:
final_model_test_perf = model_performance_regression(final_model, x_test, y_test)print("Test Performance:")final_model_test_perf

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,1.302336,0.863677,0.973067,0.973056,7.306138


  * 테스트 데이터에 대해서도 역시 모델이 좋은 성능을 보여준다. 즉, 모델이 일반화된 성능을 보여줌을 확인할 수 있다.  * RMSE와 MAE의 단위가 days 로 동일하다. 하지만 RMSE가 MAE보다 더 큰데, 이것은 아웃라이어에 더욱 패널티를 주기 때문이다.  * **MAE < 1은 모형이 평균 오차 1일 이내에 체류 기간을 예측할 수 있음을 나타냅니다** 이것은 매우 좋은 성능을 보여줍니다.  * **테스트 데이터의 MAPE 가 7.31인 것은 모델이 실제 환자의 입원 기간의 ~7% 이내로 예측할 수 있다는 것을 암시합니다.**

# Observations  1. 랜덤포레스트 리그레서 모델이 최적의 결과를 제공했습니다. 성공적으로 테스트 데이터의 98%를 포착하였습니다.  2. MAE가 0.75 인점은 모델이 성공적으로 환자의 입원 기간을 평균 오차 하루 이내로 예측할 수 있다는 것을 의미합니다.  3. 방문객과 같은 **patient** 와 **admission deposit** 요인이 예측에 중요한 역할을 수행했습니다.  4. **staff_available** 과 **extra rooms** 요인은 모델 예측으로부터 매우 적은 역할을 수행했습니다.

# Business Insights and Recommendations  * 산부인과가 병원의 가장 바쁜 부서이며, 전체 환자 수의 68.7% 를 다루고 있습니다. 부서의 원활한 기능을 위해 충분한 자원과 직원이 필요합니다.  * 환자와 동행한 방문객들의 수는 환자의 입원 기간에 많은 영향을 줍니다. 최대 입장객 수는 32명까지 가능하여 매우 높은 수치이다. 이에 대해서는 제한을 둘 수 있다.  * 환자들의 74.2% 가 여성이다. 그래서, 이 수치를 염두에 두면서 자원을 조달할 필요가 있다  * 환자들의 가장 큰 수치가 (89.3%) 트라우마 또는 긴급 입원의 경우이다. 구급차와 응급실의 증가는 사상자의 위험을 줄일 수 있다.  * 병동 A 는 가장 길게 머무르며 가장 심각한 환자들의 수치가 가장 많다. 이 병동에서는 더 많은 자원과 스태프들을 배치하여 환자들의 입원 기간을 줄일 필요가 있다.  * 나이가 많은 환자들 (51-100) 과 어린이들 (1-10) 이 가장 오래 머문다. 이러한 연령층에 대한 각별한 주의는 병원에서 더 빠른 퇴원으로 이어질 수 있습니다.  * 병동 D,E, C는 환자와 동행한 방문객들이 가장 많습니다. 이 병동은 세면실, 상점, 방문객들을 위한 로비와 같은 더 많은 공간과 어메너티들이 필요로 할 것입니다. 추가 수입을 창출하기 위해 상점 소유자와 광고에 공간을 임대할 수도 있습니다.  * 마지막으로, Random Forest Regressor가 하루의 오차로 환자들의 입원기간을 잘 예측할 수 있습니다. 병원은 이러한 예측을 사용하여 자원과 직원을 그에 맞게 할당하고 모든 종류의 낭비를 줄일 수 있습니다. 병원은 심지어 응급 상황에도 최적 입원에 따라서 병동과 의사들을 할당할 수 있습니다.

# Additional Content (Optional)# Boosting Models이제 앙상블 기법의 한 종류인 - 부스팅 기법에 대하여 살펴봅시다.# XGBoost  * XGBoost는 Extreme Gradient Boosting을 의미합니다.  * XGBoost는 트리계열의 앙상블 머신러닝 기법이며, 이 모델은 Gradient Boosting 프레임워크와 몇가지 신뢰할 수 있는 근사 알고리즘을 통합하여 예측력과 성능을 향상시킵니다. 이 모델은 데이터 사이언스 대회의 리더보드에 순위권에 자주 보이며 넓게 사용됩니다.

In [ ]:
#installing the xgboost library using "pip' command.!pip install xgboost

In [ ]:
#importing the Random Forest Regressor and Bagging Regressor [Bagging]from sklearn.ensemble import RandomForestRegressor, BaggingRegressor#importing the AdaBoostRegressor and GradientBoostingRegressor [Boosting]from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor#importing the XGBReressor from the xgboostfrom xgboost import XGBRegressor

In [ ]:
#Adaboost Regressoradaboost_model = AdaBoostRegressor(random_state = 1)#Fitting the modeladaboost_model.fit(x_train, y_train)# model Performance on test data i.e predictionadaboost_model_perf_test = model_performance_regression(adaboost_model, x_test, y_test)adaboost_model_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,2.375388,1.58689,0.910399,0.910363,13.623722


In [ ]:
#Gradient Boost Regressorgbc = GradientBoostingRegressor(random_state=1)gbc.fit(x_train,y_train)gbc_perf_test = model_performance_regression(gbc, x_test, y_test)gbc_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,1.792721,1.212749,0.948965,0.948944,10.247284


In [ ]:
#XGBoost Regressorxgb = XGBRegressor(random_state=1, eval_metric='logloss')xgb.fit(x_train,y_train)xgb_perf_test = model_performance_regression(xgb, x_test, y_test)xgb_perf_test

,RMSE,MAE,R-squard,Adj. R-squared,MAPE
0,1.513463,1.034136,0.963626,0.963612,8.868662


# Hyperparameter Tuning: Boosting하이퍼 파라미터 튜닝은 머신러닝에서 모델의 최적 파라미터를 개발하기 위한 좋은 기술입니다. 만약 데이터 사이즈가 증가한다면, 학습 프로세스 동안 계산 시간이 증가하게 될것입니다.  * 연습 목적으로, 모델 성능을 향상시키기 위해 조정되어야할 각 알고리즘의 중요한 하이퍼 파라미터들에 대하여 나열해보겠습니다.  1. Adaboost     * 몇몇 중요한 하이퍼파라미터들이 튜닝될 수 있습니다:       * **base_estimator** object, default = None, 어떤 부스팅된 앙상블이 만들어질지에 대한 기본 추정치입니다. 만약 None 이라면, base estimator 는 max_depth=3 인 DecisionTreeRegressor 로 초기화됩니다.       * **n_estimators** int, default = 50, 부스팅이 종료될 최대 추정치의 수 를 의미합니다. 완벽하게 피팅되는 경우, 학습 과정이 조기에 중단됩니다.       * **loss** : {'linear', 'square', 'exponential'}, default = 'linear' , 각 부스팅 실행 이후에 weight를 업데이트할 때 사용할 손실 함수입니다.       * **learning_rate** float, default = 1.0, 각 부스팅 실행에 적용될 regresso 가중치. 높은 learning rate 일 수록 각 regressor의 기여도를 증가시킵니다.  2. Gradient Boosting Algorithm     * 몇몇 중요한 하이퍼파라미터들이 튜닝될 수 있습니다.       * **n_estimators** : 수행될 부스팅 스테이지의 수입니다.       * **max_depth** : 트리 노드 수의 한계입니다. 최적 값은 인풋 변수의 상호작용에 의존합니다.       * **min_samples_split** : 내부의 노드를 나누기 위해 필요한 표본의 최소 수       * **learning_rate** : 각 트리의 기여도가 얼마나 줄어들지 대한 수준       * **loss** : 최적화할 손실 함수Gradient Boosting Regressor의 각 파라미터에 대해 더 나은 이해를 위해 이 [source](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor) 를 참고하세요.  3. XGBoost Algorithm     * 몇몇 중요한 하이퍼파라미터들이 튜닝될 수 있습니다.       * **booster** (default = gbtree) , 어떤 부스터를 사용할지, gbtree, gblinear, dart 가 될 수 있습니다. gbtree와 dart는 트리기반 모델을 사용하는 반면 gblinear는 선형 함수를 사용합니다.       * **min_child_weight** (default = 1) , 자식에 필요한 인스턴스 가중치(hesian)의 최소 합이다. 트리 분할 단계가 인스턴스 가중치의 합이 min_child_weight보다 작은 리프 노드로 귀결되면, 빌딩 프로세스는 더 이상의 분할을 포기할 것이다. 선형 회귀 작업에서, 이것은 단순히 각각의 노드에 있어야 하는 최소 인스턴스 수에 대응한다. min_child_weight가 클수록, 알고리즘은 더 보수적일 것이다.XGBoost Regressor 의 각 파라미터에 대해 더 나은 이해를 위해 이 [source](https://xgboost.readthedocs.io/en/stable/parameter.html) 를 참고하세요.

# 지금까지 만든 모든 모델 비교

In [ ]:
models_test_comp_df = pd.concat(    [        lin_reg_test.T,        ridge_regression_perf_test.T,        dt_regressor_perf_test.T,        regressor_perf_test.T,        bagging_estimator_perf_test.T,        dtree_tuned_regressor_perf_test.T,        bagging_tuned_regressor_perf_test.T,        rf_tuned_regressor_perf_test.T    ],    axis = 1)models_test_comp_df.columns = [    "Linear Regression",    "Ridge Regression",    "Decision tree regressor",    "Random Forest regressor",    "Bagging regressor",    "Tuned Decision Tree regressor",    "Tuned Bagging Tree regressor",    "Tuned Random Forest Regressor"]print("Test performance comparision:")models_test_comp_df

,Linear Regression,Ridge Regression,Decision tree regressor,Random Forest regressor,Bagging regressor,Adaboost regressor,Gradientboost regressor,XGBoost regressor
RMSE,3.144070,3.144057,1.821321,1.302336,1.364505,2.375388,1.792721,1.513463
MAE,2.155771,2.155826,1.131270,0.863677,0.902326,1.586890,1.212749,1.034136
R-squard,0.843027,0.843028,0.947324,0.973067,0.970434,0.910399,0.948965,0.963626
Adj. R-squared,0.842972,0.842963,0.947302,0.973056,0.970422,0.910363,0.948944,0.963612
MAPE,19.676973,19.677968,9.353216,7.306138,7.627444,13.623722,10.247284,8.868662


**Obesrvationss** :  * default 파라미터의 배깅 방법이 부스팅 방법보다 더 성능이 좋다.  * **Random Forest Regressor 가 이 데이터셋에 최적의 성능을 보여준다**